# 02 - Exploratory Data Analysis (EDA)

**Objective**: Statistical analysis and visualization of churn patterns

**Outputs**:
- Distribution plots by churn class
- Statistical tests (T-tests, Chi-square)
- Correlation heatmap with multicollinearity detection
- Top churn signal features

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', 100)
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

print('Libraries loaded!')

In [ ]:
# Load data
RAW_DATA_PATH = Path('../data/01_raw')
REPORTING_PATH = Path('../data/08_reporting')
REPORTING_PATH.mkdir(parents=True, exist_ok=True)

df = pd.read_csv(RAW_DATA_PATH / 'cell2celltrain.csv')
print(f"Loaded: {df.shape[0]:,} rows × {df.shape[1]} columns")

## 1. Univariate Analysis - Numerical Features

In [ ]:
# Key numerical features for analysis
key_numerical = [
    'MonthlyRevenue', 'MonthlyMinutes', 'MonthsInService', 
    'TotalRecurringCharge', 'DroppedCalls', 'BlockedCalls',
    'CustomerCareCalls', 'OverageMinutes'
]
key_numerical = [f for f in key_numerical if f in df.columns]

target = 'Churn'
df_churn = df[df[target] == 1]
df_retain = df[df[target] == 0]

print(f"Churners: {len(df_churn):,}")
print(f"Retained: {len(df_retain):,}")

In [ ]:
# Distribution plots overlaid by churn
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for i, col in enumerate(key_numerical[:8]):
    ax = axes[i]
    
    # Plot distributions
    sns.kdeplot(data=df_retain[col].dropna(), ax=ax, label='Retained', color='#2ecc71', fill=True, alpha=0.3)
    sns.kdeplot(data=df_churn[col].dropna(), ax=ax, label='Churned', color='#e74c3c', fill=True, alpha=0.3)
    
    ax.set_title(col, fontsize=12, fontweight='bold')
    ax.legend()

plt.tight_layout()
plt.savefig(REPORTING_PATH / 'distribution_by_churn.png', dpi=150)
plt.show()
print(f" Saved: distribution_by_churn.png")

In [ ]:
# Box plots for outlier detection
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for i, col in enumerate(key_numerical[:8]):
    ax = axes[i]
    df.boxplot(column=col, by=target, ax=ax)
    ax.set_title(col, fontsize=12, fontweight='bold')
    ax.set_xlabel('Churn')

plt.suptitle('Box Plots by Churn Status', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(REPORTING_PATH / 'boxplots_by_churn.png', dpi=150)
plt.show()

## 2. Statistical Tests (T-Tests for Numerical Features)

In [ ]:
def perform_ttest(df, feature, target='Churn'):
    """Perform independent T-test between churn groups."""
    churners = df[df[target] == 1][feature].dropna()
    retained = df[df[target] == 0][feature].dropna()
    
    statistic, pvalue = stats.ttest_ind(churners, retained, equal_var=False)
    
    return {
        'Feature': feature,
        'Mean (Churn)': churners.mean(),
        'Mean (Retain)': retained.mean(),
        'Difference': churners.mean() - retained.mean(),
        'T-Statistic': statistic,
        'P-Value': pvalue,
        'Significant': ' Yes' if pvalue < 0.05 else ' No'
    }

# Perform T-tests on all numerical features
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
numerical_cols = [c for c in numerical_cols if c != target and 'ID' not in c.upper()]

ttest_results = [perform_ttest(df, col) for col in numerical_cols]
ttest_df = pd.DataFrame(ttest_results)
ttest_df = ttest_df.sort_values('P-Value')

print(" T-TEST RESULTS (Top 20 by Significance)")
print("="*80)
display(ttest_df.head(20))

In [ ]:
# Top significant features
significant_features = ttest_df[ttest_df['P-Value'] < 0.05]['Feature'].tolist()
print(f"\n Significant features (p < 0.05): {len(significant_features)}")
print(significant_features[:15])

## 3. Churn Rate by Categorical Features

In [ ]:
# Categorical features to analyze
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
print(f"Categorical columns: {categorical_cols}")

In [ ]:
def calculate_churn_rate_by_category(df, feature, target='Churn'):
    """Calculate churn rate for each category."""
    churn_rate = df.groupby(feature)[target].agg(['mean', 'count'])
    churn_rate.columns = ['Churn Rate', 'Count']
    churn_rate['Churn Rate'] = churn_rate['Churn Rate'] * 100
    return churn_rate.sort_values('Churn Rate', ascending=False)

# Analyze key categorical features
key_categorical = ['MaritalStatus', 'Homeownership', 'IncomeGroup', 'CreditRating']
key_categorical = [c for c in key_categorical if c in df.columns]

for col in key_categorical:
    print(f"\n Churn Rate by {col}:")
    display(calculate_churn_rate_by_category(df, col))

In [ ]:
# Visualize churn rate by key categories
if len(key_categorical) > 0:
    fig, axes = plt.subplots(1, min(4, len(key_categorical)), figsize=(16, 5))
    if len(key_categorical) == 1:
        axes = [axes]
    
    for i, col in enumerate(key_categorical[:4]):
        ax = axes[i]
        churn_by_cat = df.groupby(col)[target].mean() * 100
        churn_by_cat.plot(kind='bar', ax=ax, color='#3498db', edgecolor='black')
        ax.set_title(f'Churn Rate by {col}', fontweight='bold')
        ax.set_ylabel('Churn Rate (%)')
        ax.axhline(y=df[target].mean()*100, color='red', linestyle='--', label='Overall Rate')
        ax.legend()
        plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.savefig(REPORTING_PATH / 'churn_by_category.png', dpi=150)
    plt.show()

## 4. Chi-Square Tests for Categorical Features

In [ ]:
def perform_chi_square(df, feature, target='Churn'):
    """Perform Chi-square test for independence."""
    contingency = pd.crosstab(df[feature], df[target])
    chi2, pvalue, dof, expected = stats.chi2_contingency(contingency)
    
    # Cramér's V for effect size
    n = contingency.sum().sum()
    cramers_v = np.sqrt(chi2 / (n * (min(contingency.shape) - 1)))
    
    return {
        'Feature': feature,
        'Chi-Square': chi2,
        'P-Value': pvalue,
        'Cramér\'s V': cramers_v,
        'Significant': ' Yes' if pvalue < 0.05 else ' No'
    }

# Perform chi-square tests
chi_results = []
for col in categorical_cols:
    try:
        result = perform_chi_square(df, col)
        chi_results.append(result)
    except Exception as e:
        print(f"Skipping {col}: {e}")

chi_df = pd.DataFrame(chi_results).sort_values('P-Value')

print(" CHI-SQUARE TEST RESULTS")
print("="*80)
display(chi_df)

## 5. Correlation Analysis

In [ ]:
# Calculate correlation matrix for numerical features
numerical_df = df.select_dtypes(include=['int64', 'float64'])
corr_matrix = numerical_df.corr()

# Correlation heatmap
plt.figure(figsize=(20, 16))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, cmap='RdBu_r', center=0,
            square=True, linewidths=0.5, annot=False, fmt='.2f',
            cbar_kws={'shrink': 0.8})
plt.title('Feature Correlation Heatmap', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig(REPORTING_PATH / 'correlation_heatmap.png', dpi=150)
plt.show()
print(f" Saved: correlation_heatmap.png")

In [ ]:
# Identify highly correlated pairs (|r| > 0.85)
def find_high_correlations(corr_matrix, threshold=0.85):
    """Find feature pairs with correlation above threshold."""
    high_corr = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                high_corr.append({
                    'Feature 1': corr_matrix.columns[i],
                    'Feature 2': corr_matrix.columns[j],
                    'Correlation': corr_matrix.iloc[i, j]
                })
    return pd.DataFrame(high_corr).sort_values('Correlation', ascending=False)

high_corr_pairs = find_high_correlations(corr_matrix, threshold=0.85)

print(" MULTICOLLINEARITY DETECTION (|r| > 0.85)")
print("="*80)
if len(high_corr_pairs) > 0:
    print(f"Found {len(high_corr_pairs)} highly correlated pairs:")
    display(high_corr_pairs)
    print("\n RECOMMENDATION: Consider dropping one feature from each pair.")
else:
    print(" No highly correlated feature pairs found.")

In [ ]:
# Correlation with target (Churn)
if target in numerical_df.columns:
    target_corr = corr_matrix[target].drop(target).sort_values(key=abs, ascending=False)
    
    plt.figure(figsize=(12, 8))
    colors = ['#e74c3c' if x > 0 else '#2ecc71' for x in target_corr.head(20).values]
    target_corr.head(20).plot(kind='barh', color=colors, edgecolor='black')
    plt.title('Top 20 Features Correlated with Churn', fontsize=14, fontweight='bold')
    plt.xlabel('Correlation Coefficient')
    plt.axvline(x=0, color='black', linewidth=0.5)
    plt.tight_layout()
    plt.savefig(REPORTING_PATH / 'churn_correlations.png', dpi=150)
    plt.show()
    
    print("\n TOP 10 CHURN PREDICTORS (by correlation):")
    for feat, corr in target_corr.head(10).items():
        direction = "↑ increases" if corr > 0 else "↓ decreases"
        print(f"  {feat}: {corr:.4f} ({direction} churn)")

## 6. MonthsInService Analysis (J-Curve Pattern)

In [ ]:
# Analyze churn by tenure (expecting J-curve pattern)
if 'MonthsInService' in df.columns:
    # Create tenure bins
    df['TenureBin'] = pd.cut(df['MonthsInService'], 
                             bins=[0, 3, 6, 12, 24, 36, 100],
                             labels=['0-3mo', '3-6mo', '6-12mo', '12-24mo', '24-36mo', '36+mo'])
    
    churn_by_tenure = df.groupby('TenureBin')[target].agg(['mean', 'count'])
    churn_by_tenure.columns = ['Churn Rate', 'Count']
    churn_by_tenure['Churn Rate'] = churn_by_tenure['Churn Rate'] * 100
    
    print(" CHURN BY TENURE (Looking for J-curve)")
    display(churn_by_tenure)
    
    # Visualize
    fig, ax = plt.subplots(figsize=(10, 6))
    churn_by_tenure['Churn Rate'].plot(kind='bar', ax=ax, color='#3498db', edgecolor='black')
    ax.axhline(y=df[target].mean()*100, color='red', linestyle='--', label='Overall Rate')
    ax.set_title('Churn Rate by Customer Tenure', fontsize=14, fontweight='bold')
    ax.set_ylabel('Churn Rate (%)')
    ax.set_xlabel('Tenure Segment')
    ax.legend()
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.savefig(REPORTING_PATH / 'churn_by_tenure.png', dpi=150)
    plt.show()
    
    # Drop temp column
    df.drop('TenureBin', axis=1, inplace=True)

## 7. Feature Importance (Preliminary Random Forest)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

# Prepare data for quick RF
df_rf = df.copy()

# Encode categorical variables
le = LabelEncoder()
for col in df_rf.select_dtypes(include=['object']).columns:
    df_rf[col] = df_rf[col].fillna('Unknown')
    df_rf[col] = le.fit_transform(df_rf[col].astype(str))

# Fill missing numerical values
df_rf = df_rf.fillna(df_rf.median())

# Exclude ID and leakage features
exclude_cols = ['CustomerID', 'RetentionCalls', 'RetentionOffersAccepted', 'MadeCallToRetentionTeam', target]
feature_cols = [c for c in df_rf.columns if c not in exclude_cols]

X = df_rf[feature_cols]
y = df_rf[target]

print(f"Training Quick RF on {X.shape[1]} features...")

In [ ]:
# Train quick Random Forest (non-tuned baseline)
rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
rf.fit(X, y)

# Feature importance
importance_df = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': rf.feature_importances_
}).sort_values('Importance', ascending=False)

# Visualize top 20
plt.figure(figsize=(12, 8))
importance_df.head(20).sort_values('Importance').plot(
    kind='barh', x='Feature', y='Importance', 
    color='#9b59b6', edgecolor='black', legend=False
)
plt.title('Top 20 Features (Random Forest Importance)', fontsize=14, fontweight='bold')
plt.xlabel('Importance')
plt.tight_layout()
plt.savefig(REPORTING_PATH / 'rf_feature_importance.png', dpi=150)
plt.show()

print("\n TOP 15 FEATURES BY RF IMPORTANCE:")
for _, row in importance_df.head(15).iterrows():
    print(f"  {row['Feature']}: {row['Importance']:.4f}")

## 8. Summary & Insights

In [ ]:
# Save key findings
findings = {
    'significant_ttest_features': ttest_df[ttest_df['P-Value'] < 0.05]['Feature'].tolist()[:20],
    'significant_chi_features': chi_df[chi_df['P-Value'] < 0.05]['Feature'].tolist() if len(chi_df) > 0 else [],
    'multicollinear_pairs': high_corr_pairs.to_dict('records') if len(high_corr_pairs) > 0 else [],
    'top_rf_features': importance_df.head(20)['Feature'].tolist()
}

# Save to CSV
ttest_df.to_csv(REPORTING_PATH / 'ttest_results.csv', index=False)
if len(chi_df) > 0:
    chi_df.to_csv(REPORTING_PATH / 'chi_square_results.csv', index=False)
importance_df.to_csv(REPORTING_PATH / 'rf_importance.csv', index=False)

print("="*70)
print(" EDA SUMMARY")
print("="*70)
print(f"\n T-Test significant features: {len(findings['significant_ttest_features'])}")
print(f" Chi-Square significant features: {len(findings['significant_chi_features'])}")
print(f" Multicollinear pairs (r>0.85): {len(findings['multicollinear_pairs'])}")
print(f"\n Top 5 Churn Predictors (RF):")
for feat in findings['top_rf_features'][:5]:
    print(f"   • {feat}")

print("\n" + "="*70)
print(" NEXT: Proceed to 03_Preprocessing.ipynb")
print("="*70)